$$ sync = (0.5 + 0.5 * square) * modRange + modMin $$

In [ ]:
from __future__ import division
syncLen = 39
spaceLen = 47
videoLen = 909
telemetryLen = 45
lineLen = 2080

rate = 4160
T = 1/rate

# modulation index 95 % 
modMax = 244 / 256
modMin = 11 / 256
modRange = (244-11) / 256

from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

mul = 8000
time = np.linspace(0.0,1,mul)

# 1040 Hz 95 % modulating square wave
syncA = signal.square(2*np.pi* 1040 *time, 0.5)
# move it between 0..1
syncA = [0.5 + 0.5*val for val in syncA]
# move it between 
syncA = [modRange * val + modMin for val in syncA]


plt.plot(time, syncA)

#time = linspace(0, 1, 8000)
plt.grid()
plt.ylim(0, 1)
plt.xlim(0, syncLen*T)



In [36]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import math
import random

def syncA():
    #Async pre-wait 4 T
    data = mul*4*[11]
    # 7 1040 Hz 50% duty pulses
    for i in xrange(0,7):
        data += mul*2 * [244]
        data += mul*2*[11]
    data += mul*7 * [11]
    return data

def syncB():
        # syncB pre-wait 4 T
    data = mul*4*[11]
    # 7 832 Hz 3/5 duty pulses
    for i in xrange(0,7):
        data += mul*3 * [244]
        data += mul*2*[11]
    return data

def markerA():
    return mul*47 * [11]
def markerB():
    return mul*47 * [244]
def videoA():
    data = 0*[0]
    for i in range(0,mul*909):
        data += [int((i*(244-11))/(mul*909) + 11)]
    return data
def videoB():
    data = mul*909*[0]
    data = [random.randint(11,244) for val in data]
    return data

def telemetryA():
    return mul*45 * [128]
def telemetryB():
    return mul*45 * [128]

def line():
    data = syncA()
    data += markerA()
    data += videoA()
    data += telemetryA()
    data += syncB()
    data += markerB()
    data += videoB()
    data += telemetryB()
    return data

def frame():
    data = [0]*0
    for i in xrange(0,16):
        data += line()
    return data

In [37]:
mul = 1

data = line()
#data = signal.decimate(data, mul)
idx = np.linspace(0,len(data) -1 ,len(data))

import sys

print len(idx)
print len(data)

#subcarrier = [data * np*sin(np.pi*2*2400)]
print(len(syncA()))
print(len(markerA()))
print(len(videoA()))
print(len(telemetryA()))
print(len(syncB()))
print(len(markerB()))
print(len(videoB()))
print(len(telemetryB()))

total = len(syncA())+len(markerA())+len(videoA())+len(telemetryA())+len(syncB())+len(markerB())+len(videoB())+len(telemetryB())
print(total)
#plt.step(idx, data)
#plt.plot(idx, data)
#plt.show()
print 2080 * mul

2080
2080
39
47
909
45
39
47
909
45
2080
2080


In [17]:
# data signal for creating a bitmap
mul = 1
data = line()
idx = np.linspace(0,len(data) -1 ,len(data))
plt.scatter(idx, data)
plt.show()